In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población media anual,513170
1,Huelva,Confirmados PDIA,53482
2,Huelva,Confirmados PDIA 14 días,4518
3,Huelva,Tasa PDIA 14 días,"880,4100005846016"
4,Huelva,Confirmados PDIA 7 días,2465
5,Huelva,Tasa PDIA 7 dias,"480,3476430812401"
6,Huelva,Total Confirmados,53693
7,Huelva,Curados,47230
8,Huelva,Fallecidos,426


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  53482.0


/tmp/ipykernel_34509/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  15153.0


/tmp/ipykernel_34509/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población media anual']['Valor'].values[0]

/tmp/ipykernel_34509/242626003.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_34509/242626003.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_34509/242626003.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población media anual']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 176 personas en los últimos 7 días 

Un positivo PCR cada 96 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población media anual' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [13]:
# Rename columns
tabla.columns = ['Confirmados PDIA', 'Confirmados PDIA 14 días', 'Confirmados PDIA 7 días', 'Población']

In [14]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [15]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [16]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

,Confirmados PDIA,Confirmados PDIA 14 días,Confirmados PDIA 7 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,53482.0,4518.0,2465.0,513170.0,480.347643,880.410001,504.0
Huelva-Costa,31539.0,2644.0,1460.0,289548.0,504.234186,913.147388,215.0
Condado-Campiña,16322.0,1079.0,576.0,156231.0,368.684832,690.643982,157.0
Sierra de Huelva-Andévalo Central,5186.0,789.0,426.0,67391.0,632.131887,1170.779481,133.0
Huelva (capital),15153.0,1497.0,816.0,143837.0,567.308829,1040.761417,123.0
Valverde del Camino,1112.0,183.0,143.0,12750.0,1121.568627,1435.294118,31.0
Aracena,968.0,219.0,82.0,8255.0,993.337371,2652.937614,27.0
Ayamonte,2397.0,250.0,141.0,21104.0,668.119788,1184.609553,23.0
Escacena del Campo,280.0,38.0,25.0,2287.0,1093.135111,1661.565369,18.0


In [17]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [18]:
tabla.sort_values('tasa14', ascending=False)

,Confirmados PDIA,Confirmados PDIA 14 días,Confirmados PDIA 7 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
San Silvestre de Guzmán,59.0,34.0,30.0,614.0,4885.993485,5537.459283,0.0
Fuenteheridos,39.0,21.0,10.0,709.0,1410.437236,2961.918195,3.0
Aracena,968.0,219.0,82.0,8255.0,993.337371,2652.937614,27.0
Puerto Moral,21.0,7.0,2.0,278.0,719.424460,2517.985612,0.0
Cumbres Mayores,202.0,36.0,14.0,1749.0,800.457404,2058.319039,6.0
Escacena del Campo,280.0,38.0,25.0,2287.0,1093.135111,1661.565369,18.0
Galaroza,78.0,22.0,14.0,1382.0,1013.024602,1591.895803,9.0
Villanueva de las Cruces,30.0,6.0,3.0,387.0,775.193798,1550.387597,3.0
Almonaster la Real,121.0,26.0,10.0,1788.0,559.284116,1454.138702,5.0


In [19]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

,Confirmados PDIA,Confirmados PDIA 14 días,Confirmados PDIA 7 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Corteconcepción,57.0,6.0,1.0,536.0,186.567164,1119.402985,0.0,0.166667
Campillo (El),122.0,21.0,4.0,2024.0,197.628458,1037.549407,1.0,0.190476
Villarrasa,253.0,12.0,3.0,2211.0,135.685210,542.740841,0.0,0.250000
Puerto Moral,21.0,7.0,2.0,278.0,719.424460,2517.985612,0.0,0.285714
Alájar,44.0,9.0,3.0,759.0,395.256917,1185.770751,1.0,0.333333
Villalba del Alcor,522.0,17.0,6.0,3366.0,178.253119,505.050505,1.0,0.352941
Bollullos Par del Condado,1562.0,133.0,49.0,14387.0,340.585251,924.445680,10.0,0.368421
Aracena,968.0,219.0,82.0,8255.0,993.337371,2652.937614,27.0,0.374429
Manzanilla,182.0,29.0,11.0,2118.0,519.357885,1369.216242,6.0,0.379310
